### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "knn_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [156]:
import numpy as np

In [224]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
cl = KNeighborsClassifier(n_neighbors=5,weights='distance')
train = np.array([[1,4,3],[3,7,4],[1,0,8]])
cl.fit(np.array([[1,2,3],[1,5,3],[90,2,3],[3,2,6],[5,4,3]]), np.array([0,1,1,2,0]))
pr = cl.predict(train)
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(np.array([[1,2,3],[1,5,3],[90,2,3],[3,2,6],[5,4,3]]), np.array([0,1,1,2,0]))
a = cl.kneighbors(train,3)
b = model.kneighbors(train,3)
print(a,b)

(array([[1.        , 2.        , 4.        ],
       [3.        , 3.74165739, 5.38516481],
       [3.46410162, 5.38516481, 7.07106781]]), array([[1, 0, 4],
       [1, 4, 3],
       [3, 0, 1]], dtype=int64)) (array([[1.        , 2.        , 4.        ],
       [3.        , 3.74165739, 5.38516481],
       [3.46410162, 5.38516481, 7.07106781]]), array([[1, 0, 4],
       [1, 4, 3],
       [3, 0, 1]], dtype=int64))


In [218]:
class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям. 
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    def __init__(self, n_neighbors=1, weights='uniform', metric="l2"):
        self.k_n = n_neighbors
        self.weight = weights
        self.p = metric
     
    def fit(self, x, y):
        self.train = x.copy()
        self.answers = y.copy()
        return self
        '''Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
    def predict_one(self, x):
        classes = np.unique(self.answers)
        return classes[np.argmax(self.predict_proba_one(x))]
    def predict(self, x):
        return np.apply_along_axis(self.predict_one,1,x)
        
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
    def predict_proba_one(self, x):
        k_nearest = self.answers[self.kneighbors_one_inds(x,self.k_n)]
        classes = np.unique(self.answers)
        if self.weight=='distance':
            k_nearest_dist = self.kneighbors_one_dists(x,self.k_n)
            weigh = lambda x: (1/k_nearest_dist[np.where(k_nearest==x)[0]]).sum()
            vweigh = np.vectorize(weigh)
            return vweigh(classes)/(vweigh(classes).sum())
        else:
            weigh = lambda x: np.ones(self.k_n)[np.where(k_nearest==x)[0]].sum()
            vweigh = np.vectorize(weigh)
            return vweigh(classes)/(vweigh(classes).sum())
    def predict_proba(self, X):
        return np.apply_along_axis(self.predict_proba_one,1,X)
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности 
        объекта к каждому классу
        """
    def kneighbors_one_dists(self, x, n_neighbors):
        distances = np.sqrt(((self.train - x)**2).sum(axis=1))
        indsort = np.argsort(distances)[:n_neighbors] 
        return distances[indsort]
    def kneighbors_one_inds(self, x, n_neighbors):
        distances = np.sqrt(((self.train - x)**2).sum(axis=1))
        indsort = np.argsort(distances)[:n_neighbors] 
        return indsort
    def kneighbors(self, x, n_neighbors):
        dists = np.apply_along_axis(self.kneighbors_one_dists,1,x,n_neighbors)
        inds = np.apply_along_axis(self.kneighbors_one_inds,1,x,n_neighbors)
        return dists,inds 
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """

In [219]:
def load_file(filename):
    return np.load(filename, allow_pickle=True)[()]
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """

In [220]:
input_filename = "knn_data_043.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)

In [221]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
cl = KNeighborsClassifier(n_neighbors=5)
cl.fit(data_dict["X_train"], data_dict["y_train"])
pr=cl.predict(data_dict["X_test"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])
acc = accuracy_score(pr,l2_uniform_n5_y_predict)
print(acc)

1.0


In [222]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
cl = KNeighborsClassifier(n_neighbors=10)
cl.fit(data_dict["X_train"], data_dict["y_train"])
pr=cl.predict(data_dict["X_test"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])
acc = accuracy_score(pr,l2_uniform_10_y_predict)
print(acc)

1.0


In [223]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
cl = KNeighborsClassifier(n_neighbors=5, weights='distance')
cl.fit(data_dict["X_train"], data_dict["y_train"])
pr=cl.predict(data_dict["X_test"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])
acc = accuracy_score(pr,l2_distance_n5_y_predict)
print(acc)

1.0


In [164]:
output_filename = "results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)